In [0]:
import os
from databricks import sql
from databricks.sdk.core import Config
import streamlit as st
import pandas as pd
import altair as alt

# Ensure environment variable is set correctly
assert os.getenv('DATABRICKS_WAREHOUSE_ID'), "DATABRICKS_WAREHOUSE_ID must be set in app.yaml."
def sqlQuery(query: str) -> pd.DataFrame:
    """
    Helper function to run a SQL query against Databricks SQL Warehouse
    and return the result as a Pandas DataFrame.
    """
    cfg = Config()
    with sql.connect(
        server_hostname=cfg.host,
        http_path=f"/sql/1.0/warehouses/{os.getenv('DATABRICKS_WAREHOUSE_ID')}",
        credentials_provider=lambda: cfg.authenticate
    ) as connection:
        with connection.cursor() as cursor:
            cursor.execute(query)
            return cursor.fetchall_arrow().to_pandas()

@st.cache_data(ttl=30)
def get_model_evaluation() -> pd.DataFrame:
    """
    Fetches the recommendation model evaluation data from Unity Catalog.
    Example table path: ecommerces_catalog.recommendation_schema.model_evaluation
    """
    query = """
    SELECT 
        model_type,
        top_k,
        precision_at_k,
        recall_at_k,
        evaluation_timestamp
    FROM ecommerces_catalog.recommendation_schema.model_evaluation
    """
    return sqlQuery(query)

# -------------------------------------------------------------
# Streamlit App
# -------------------------------------------------------------
st.set_page_config(layout="wide")
st.title("Recommendation Model Evaluation Metrics V1")

eval_data = get_model_evaluation()

if eval_data.empty:
    st.warning("No evaluation metrics found in the table.")
    st.stop()
# Display raw data in a table
st.dataframe(eval_data, use_container_width=True)

# ------------------------------------------------------------------------------
# 1) Highlight which model has the highest Precision and Recall
# ------------------------------------------------------------------------------
max_precision_idx = eval_data["precision_at_k"].idxmax()
max_precision_model = eval_data.loc[max_precision_idx, "model_type"]
max_precision_value = eval_data.loc[max_precision_idx, "precision_at_k"]

max_recall_idx = eval_data["recall_at_k"].idxmax()
max_recall_model = eval_data.loc[max_recall_idx, "model_type"]
max_recall_value = eval_data.loc[max_recall_idx, "recall_at_k"]

st.subheader("Best-Performing Models")
st.markdown(f"- **Highest Precision**: {max_precision_model} (`{max_precision_value:.4f}`)")
st.markdown(f"- **Highest Recall**: {max_recall_model} (`{max_recall_value:.4f}`)")

# ------------------------------------------------------------------------------
# 2) Grouped Bar Chart for Precision & Recall
# ------------------------------------------------------------------------------
# Convert wide data (precision_at_k / recall_at_k) to long format for easier plotting
melted = eval_data.melt(
    id_vars=["model_type", "top_k", "evaluation_timestamp"],
    value_vars=["precision_at_k", "recall_at_k"],
    var_name="metric",
    value_name="value"
)

# Create a grouped bar chart in Altair
chart = (
    alt.Chart(melted)
    .mark_bar()
    .encode(
        x=alt.X("model_type:N", title="Model Type", sort=None),
        y=alt.Y("value:Q", title="Score"),
        color=alt.Color("metric:N", title="Metric", scale=alt.Scale(scheme="category10")),
        column=alt.Column("top_k:O", title="K", header=alt.Header(labelOrient="bottom")),
        tooltip=["model_type", "top_k", "metric", "value"]
    )
    .properties(width=250, height=350)
)

st.subheader("Precision and Recall by Model (Grouped Bar Chart)")
st.altair_chart(chart, use_container_width=True)